# Prepare Annotation Tables

In [119]:
import pandas as pd
import os
import math
import Bio
from Bio import Seq
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.Data.CodonTable
table = Bio.Data.CodonTable.ambiguous_dna_by_id[1]

### Get variant Data

In [224]:
variants = pd.read_csv('../4-Variation/variant_list.05.tsv', sep='\t')
variants_no_filter = pd.read_csv('../4-Variation/variant_list.tsv', sep='\t')

In [225]:
synonymous=variants[(variants.EFFECT!='SYNONYMOUS_CODING') & (variants.EFFECT!='.')]

In [226]:
synonymous.GENE.unique()

array(['ORF3a', 'S', 'orf1ab', 'N', 'ORF7a', 'ORF8', 'E', 'ORF10', 'M',
       'ORF6', 'ORF7b'], dtype=object)

In [227]:
synonymous.groupby('POS')['ALT'].value_counts()


POS    ALT
269    T      1
275    T      1
285    T      4
308    A      2
312    A      1
             ..
29516  A      1
29580  A      1
29612  C      1
29625  T      1
29642  T      3
Name: ALT, Length: 1161, dtype: int64

In [228]:
synonymous['NbSample']=0
synonymous['MeanAF']=0
groups=synonymous.groupby('POS')['ALT'].value_counts()
meanAF=synonymous.groupby(['POS','ALT'])['AF'].mean()
for i,row in synonymous.iterrows():
    synonymous.loc[i, 'NbSample']=groups[row['POS']][row['ALT']]
    synonymous.loc[i, 'MeanAF']=meanAF[row['POS']][row['ALT']]

<ipython-input-228-950837d7ae45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['NbSample']=0
<ipython-input-228-950837d7ae45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  synonymous['MeanAF']=0
/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [229]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
2,SRR11397714,NC_045512,25563,G,T,119,1.000000,0,"0,0,56,63",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
3,SRR11494669,NC_045512,25563,G,T,160,1.000000,0,"0,0,74,86",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
4,SRR11494527,NC_045512,25563,G,T,137,1.000000,0,"0,0,57,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
5,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
6,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,cTc/cCc,S,1,0.050360
4141,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Tct/Cct,S,1,0.050314
4142,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Ggc/Tgc,S,1,0.050279
4143,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gtt/Ttt,S,1,0.050279


### Get Variant Selection Data

In [230]:
selection_data = pd.read_csv('Selective_Analysis_Data/20200418.csv', sep=',')

In [231]:
selection_data

,First Detected,Site,Categories,Site composition/Evolutionary Credibility,Substitutions at <br> Internal Branches <br> [Syn:Non-syn]
0,"[[""20200403"",""20200408"",""20200410"",""20200412"",...",N 13<br/>Genomic <a target = '_blank_' href = ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,3]"
1,"[[""20200325"",""20200330"",""20200403"",""20200404""],9]",ORF3a 57<br/>Genomic <a target = '_blank_' hre...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span title = ''>Q<sub>7872</s...","[0,2]"
2,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 265<br>nsp2 85<br/>Genomic <a target = '...,"{""fel"":true,""meme"":true,""fraction"":0.993106172...","{""composition"":""<span style=\""color:#ff8c00\""t...","[0,4]"
3,"[[""20200408"",""20200410"",""20200412"",""20200414"",...",ORF1a 3353<br>3C-like proteinase 90<br/>Genomi...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,3]"
4,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1a 3606<br>nsp6 37<br/>Genomic <a target = ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:#c2a90f\""t...","[0,3]"
...,...,...,...,...,...
121,"[[""20200325"",""20200330"",""20200403"",""20200404"",...",ORF1b 1022<br>helicase 100<br/>Genomic <a targ...,"{""fel"":true,""meme"":true,""fraction"":0.99999999,...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]"
122,"[[""20200415""],9]",ORF1b 1259<br>helicase 337<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.15142841...","{""composition"":""<span style=\""color:darkgreen\...","[2,0]"
123,"[[""20200410"",""20200412"",""20200414"",""20200415""],9]",ORF1b 1270<br>helicase 348<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.99505757...","{""composition"":""<span style=\""color:darkgreen\...","[0,1]"
124,"[[""20200414"",""20200415""],9]",ORF1b 1443<br>helicase 521<br/>Genomic <a targ...,"{""fel"":true,""meme"":false,""fraction"":0.21441899...","{""composition"":""<span style=\""color:darkgreen\...","[0,0]"


Clean up of data, split collumns and removing html tags

In [232]:
selection_data['syn']=0
selection_data['non_syn']=0
selection_data['REF']=0
selection_data['ALT']=0
selection_data['Genomic_coord_end']=0
selection_data['Genomic_coord_start']=0
selection_data['Gene']=0
selection_data['ref_codon']=0
selection_data['alt_codon']=0
selection_data['ref_residue']=0
selection_data['alt_residue']=0
selection_data['NbSample']=0
selection_data['MeanAF']=0

In [233]:
for i,row in selection_data.iterrows():
    selection_data.loc[i,'syn'] =row['Substitutions at <br> Internal Branches <br> [Syn:Non-syn]'].replace('[','').replace(']','').split(',')[0]
    selection_data.loc[i, 'non_syn']=row['Substitutions at <br> Internal Branches <br> [Syn:Non-syn]'].replace('[','').replace(']','').split(',')[1]
    selection_data.loc[i, 'Genomic_coord_start']=selection_data['Site'][i][:-4].split('>')[-1].split('-')[0]
    selection_data.loc[i, 'Genomic_coord_end']=selection_data['Site'][i][:-4].split('>')[-1].split('-')[1]
    selection_data.loc[i, 'NbSample']=len(row['First Detected'].replace('[[','').replace("],9]",'').split(','))
    locations=selection_data['Site'][i][:-4].split('<br/>')[0].split('<br>')
    selection_data.loc[i, 'Gene']=",".join(locations)
    composition=selection_data['Site composition/Evolutionary Credibility'][i].split(',')[2:]
    if len(composition)>0:
        variation=list((np.unique(np.array([i.replace('"',"") for i in composition if '/' in i]))))
        selection_data.loc[i, 'ref_codon']=list(variation)[0].split('/')[0]
        selection_data.loc[i, 'alt_codon']=','.join([i.split('/')[1] for i in variation])
        selection_data.loc[i, 'ref_residue']=Bio.Seq._translate_str(list(variation)[0].split('/')[0], table)
        selection_data.loc[i, 'alt_residue']=','.join([Bio.Seq._translate_str(i.split('/')[1], table) for i in variation])
    else :
        variation=selection_data['Site composition/Evolutionary Credibility'][i].replace('{"composition":"','').replace("<span title = \'\'>",'').replace('</sub></span>","variants":[]}','').split('</sub></span>')
        dic={}
        variants=''
        for j in variation:
            var=re.sub("\<span.*\\'\>",'',j)
            dic[var.split('<sub>')[0]]=var.split('<sub>')[1] 
            variants=variants+var.split('<sub>')[0]+','
        selection_data.loc[i, 'alt_residue']=variants
    
 

Remove unecessary columns

In [234]:
smaller=selection_data[['syn','non_syn','REF','ALT','Genomic_coord_end','Genomic_coord_start','Gene','ref_codon','alt_codon','ref_residue','alt_residue','NbSample','MeanAF']]

In [235]:
smaller

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,Gene,ref_codon,alt_codon,ref_residue,alt_residue,NbSample,MeanAF
0,0,3,0,0,28312,28310,N 13,Cgc,Ggc,R,G,6,0
1,0,2,0,0,25563,25561,ORF3a 57,caG,"caA,caT",Q,"Q,H",4,0
2,0,4,0,0,1060,1058,"ORF1a 265,nsp2 85",acC,acT,T,T,9,0
3,0,3,0,0,10324,10322,"ORF1a 3353,3C-like proteinase 90",aaG,aaT,K,N,5,0
4,0,3,0,0,11083,11081,"ORF1a 3606,nsp6 37",ttG,ttT,L,F,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0,1,0,0,16536,16534,"ORF1b 1022,helicase 100",0,0,0,"S,I,G,",9,0
122,2,0,0,0,17247,17245,"ORF1b 1259,helicase 337",0,0,0,"R,",1,0
123,0,1,0,0,17280,17278,"ORF1b 1270,helicase 348",0,0,0,"V,L,",4,0
124,0,0,0,0,17799,17797,"ORF1b 1443,helicase 521",0,0,0,"V,",2,0


## Add Feature data  

Main Feature , eg. ORF1a, and secondary Feature, eg. nsp5

In [236]:
smaller['MainFeat']='NA'
smaller['MainResidue']=0
smaller['SecondFeat']='NA'
smaller['SecondResidue']=0

<ipython-input-236-cba309822c72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['MainFeat']='NA'
<ipython-input-236-cba309822c72>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smaller['MainResidue']=0
<ipython-input-236-cba309822c72>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [237]:
for i,row in smaller.iterrows():
    liste_loc=row['Gene'].split(',')
    smaller.loc[i, 'MainFeat']=" ".join(liste_loc[0].split(" ")[:-1])
    smaller.loc[i, 'MainResidue']=liste_loc[0].split(" ")[-1]
    if len(liste_loc)>1:   
        smaller.loc[i, 'SecondFeat']=" ".join(liste_loc[1].split(" ")[:-1])
        smaller.loc[i, 'SecondResidue']=liste_loc[1].split(" ")[-1]

/Users/delphinel/anaconda3/envs/jupyter/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [238]:
workingtable=smaller.drop(labels='Gene', axis=1)

In [239]:
workingtable.MainFeat.unique()

array(['N', 'ORF3a', 'ORF1a', 'ORF1b', 'S', 'ORF7a', 'ORF8', 'M', 'ORF6'],
      dtype=object)

In [240]:
dic={}

In [241]:
for i in workingtable['MainFeat'].unique():
    dic[i]=workingtable[workingtable['MainFeat']==i]

In [242]:
dic['ORF1b']

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,ref_codon,alt_codon,ref_residue,alt_residue,NbSample,MeanAF,MainFeat,MainResidue,SecondFeat,SecondResidue
5,0,2,0,0,13731,13729,gCt,gTt,A,V,4,0,ORF1b,87,RdRp,97
6,0,2,0,0,14409,14407,cCt,cTt,P,L,8,0,ORF1b,313,RdRp,323
7,0,2,0,0,14787,14785,gCt,gTt,A,V,3,0,ORF1b,439,RdRp,449
31,0,1,0,0,13629,13627,Gac,Tac,D,Y,7,0,ORF1b,53,RdRp,63
32,0,1,0,0,15279,15277,Cac,Tac,H,Y,7,0,ORF1b,603,RdRp,613
62,0,3,0,0,13515,13513,0,0,0,"G,Y,C,",3,0,ORF1b,15,RdRp,25
63,0,1,0,0,17127,17125,0,0,0,"L,P,F,I,",1,0,ORF1b,1219,helicase,297
64,0,1,0,0,18654,18652,Cgc,Tgc,R,C,1,0,ORF1b,1728,3'-to-5' exonuclease,205
65,0,1,0,0,18687,18685,Tgc,Cgc,C,R,9,0,ORF1b,1739,3'-to-5' exonuclease,216
119,1,0,0,0,13947,13945,0,0,0,"P,",5,0,ORF1b,159,RdRp,169


## Import Feature data from NCBI Genbank File

In [243]:
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
from Bio import SeqIO


In [244]:
recs = [rec for rec in SeqIO.parse("annotations.gb", "genbank")]


Get Features in a Dataframe, and add ORF1b that is not in the GenBank File

In [245]:
features_list=[{'gene': 'ORF1b polyprotein', 'start': 13467, 'end': 21555}]
for rec in recs:
    feats = [feat for feat in rec.features if feat.type == "mat_peptide"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)
    feats = [feat for feat in rec.features if feat.type == "CDS"]
    for feat in feats:
        product=feat.qualifiers['product'][0]
        start=int(feat.location.start)
        end=int(feat.location.end)
        res={'gene':product,'start':start,'end':end}
        features_list.append(res)

In [246]:
features_mat=pd.DataFrame(features_list)

In [247]:
features_mat=features_mat.drop_duplicates().sort_values(by=['start']).reset_index()[['gene','start','end']]

In [248]:
features_mat['short']='na'
for i,row in features_mat.iterrows():
    features_mat.loc[i, 'short']=row['gene'].split(' ')[0]



In [249]:
features_mat

,gene,start,end,short
0,leader protein,265,805,leader
1,ORF1ab polyprotein,265,21555,ORF1ab
2,ORF1a polyprotein,265,13483,ORF1a
3,nsp2,805,2719,nsp2
4,nsp3,2719,8554,nsp3
5,nsp4,8554,10054,nsp4
6,3C-like proteinase,10054,10972,3C-like
7,nsp6,10972,11842,nsp6
8,nsp7,11842,12091,nsp7
9,nsp8,12091,12685,nsp8


Removing ORF1ab From the table

In [250]:
features_mat=features_mat.drop(labels=1, axis=0)

In [251]:
workingtable['MainFeat'].unique()

array(['N', 'ORF3a', 'ORF1a', 'ORF1b', 'S', 'ORF7a', 'ORF8', 'M', 'ORF6'],
      dtype=object)

In [252]:
list(workingtable['SecondFeat'].unique())

['NA',
 'nsp2',
 '3C-like proteinase',
 'nsp6',
 'RdRp',
 'nsp3',
 'nsp4',
 'leader protein',
 'helicase',
 "3'-to-5' exonuclease",
 'nsp7',
 'nsp8']

## Add Feature information to variant tables

In [253]:
workingtable.head(5)

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,ref_codon,alt_codon,ref_residue,alt_residue,NbSample,MeanAF,MainFeat,MainResidue,SecondFeat,SecondResidue
0,0,3,0,0,28312,28310,Cgc,Ggc,R,G,6,0,N,13,NA,0
1,0,2,0,0,25563,25561,caG,"caA,caT",Q,"Q,H",4,0,ORF3a,57,NA,0
2,0,4,0,0,1060,1058,acC,acT,T,T,9,0,ORF1a,265,nsp2,85
3,0,3,0,0,10324,10322,aaG,aaT,K,N,5,0,ORF1a,3353,3C-like proteinase,90
4,0,3,0,0,11083,11081,ttG,ttT,L,F,9,0,ORF1a,3606,nsp6,37


In [254]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,FUNCLASS,EFFECT,GENE,CODON,type,NbSample,MeanAF
2,SRR11397714,NC_045512,25563,G,T,119,1.000000,0,"0,0,56,63",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
3,SRR11494669,NC_045512,25563,G,T,160,1.000000,0,"0,0,74,86",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
4,SRR11494527,NC_045512,25563,G,T,137,1.000000,0,"0,0,57,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
5,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
6,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,ORF3a,caG/caT,S,90,0.965740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,cTc/cCc,S,1,0.050360
4141,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,orf1ab,Tct/Cct,S,1,0.050314
4142,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Ggc/Tgc,S,1,0.050279
4143,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,MISSENSE,NON_SYNONYMOUS_CODING,S,Gtt/Ttt,S,1,0.050279


In [255]:
synonymous['syn']=0
synonymous['non_syn']=1
synonymous['ref_codon']='na'
synonymous['alt_codon']='na'
synonymous['ref_residue']='na'
synonymous['alt_residue']='na'
synonymous['Genomic_coord_start']='na'
synonymous['Genomic_coord_end']='na'
synonymous['MainResidue']='na'
synonymous['MainFeat']='na'
synonymous['SecondFeat']='na'
synonymous['SecondResidue']='na'

for i,row in synonymous.iterrows():
    synonymous.loc[i, 'ref_codon']=row['CODON'].split('/')[0]
    synonymous.loc[i, 'alt_codon']=row['CODON'].split('/')[1]
    synonymous.loc[i, 'ref_residue']=Bio.Seq._translate_str(row['CODON'].split('/')[0], table)
    synonymous.loc[i, 'alt_residue']=','.join([Bio.Seq._translate_str(row['CODON'].split('/')[1], table) for i in row['CODON'].split(',')])
    features=list(features_mat[(features_mat.start<row['POS']) & (features_mat.end>row['POS'])]['gene'])
    if len(features)>1 :
        synonymous.loc[i, 'MainFeat']=features[0]
        synonymous.loc[i, 'SecondFeat']=features[1]
        startmain=int(features_mat[features_mat['gene']==features[0]].start)
        startsecond=int(features_mat[features_mat['gene']==features[1]].start)
        synonymous.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)
        synonymous.loc[i, 'SecondResidue']=math.ceil((row['POS']-(startsecond))/3)
    else :
        synonymous.loc[i, 'MainFeat']=features[0]
        startmain=int(features_mat[features_mat['gene']==features[0]].start)
        synonymous.loc[i, 'MainResidue']=math.ceil((row['POS']-(startmain))/3)
    if (row['POS']-startmain)%3==0:
        startpos=row['POS']-2
    else :
        startpos=row['POS']+1-((row['POS']-startmain)%3)
    synonymous.loc[i, 'Genomic_coord_start']=startpos
    synonymous.loc[i, 'Genomic_coord_end']=startpos+2

In [256]:
synonymous

,Sample,CHROM,POS,REF,ALT,DP,AF,SB,DP4,IMPACT,...,ref_codon,alt_codon,ref_residue,alt_residue,Genomic_coord_start,Genomic_coord_end,MainResidue,MainFeat,SecondFeat,SecondResidue
2,SRR11397714,NC_045512,25563,G,T,119,1.000000,0,"0,0,56,63",MODERATE,...,caG,caT,Q,H,25561,25563,57,ORF3a protein,na,na
3,SRR11494669,NC_045512,25563,G,T,160,1.000000,0,"0,0,74,86",MODERATE,...,caG,caT,Q,H,25561,25563,57,ORF3a protein,na,na
4,SRR11494527,NC_045512,25563,G,T,137,1.000000,0,"0,0,57,80",MODERATE,...,caG,caT,Q,H,25561,25563,57,ORF3a protein,na,na
5,SRR11494644,NC_045512,25563,G,T,203,1.000000,0,"0,0,111,92",MODERATE,...,caG,caT,Q,H,25561,25563,57,ORF3a protein,na,na
6,SRR11494701,NC_045512,25563,G,T,149,1.000000,0,"0,0,69,80",MODERATE,...,caG,caT,Q,H,25561,25563,57,ORF3a protein,na,na
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,SRR11410542,NC_045512,11751,T,C,139,0.050360,1,"35,97,1,6",MODERATE,...,cTc,cCc,L,P,11750,11752,3829,ORF1a polyprotein,nsp6,260
4141,SRR11494532,NC_045512,19159,T,C,159,0.050314,1,"58,93,4,4",MODERATE,...,Tct,Cct,S,P,19159,19161,1898,ORF1b polyprotein,3'-to-5' exonuclease,374
4142,SRR11494664,NC_045512,24131,G,T,179,0.050279,1,"81,89,5,4",MODERATE,...,Ggc,Tgc,G,C,24131,24133,857,surface glycoprotein,na,na
4143,SRR11494664,NC_045512,23903,G,T,179,0.050279,0,"79,91,4,5",MODERATE,...,Gtt,Ttt,V,F,23903,23905,781,surface glycoprotein,na,na


In [257]:
synonymous.columns

Index(['Sample', 'CHROM', 'POS', 'REF', 'ALT', 'DP', 'AF', 'SB', 'DP4',
       'IMPACT', 'FUNCLASS', 'EFFECT', 'GENE', 'CODON', 'type', 'NbSample',
       'MeanAF', 'syn', 'non_syn', 'ref_codon', 'alt_codon', 'ref_residue',
       'alt_residue', 'Genomic_coord_start', 'Genomic_coord_end',
       'MainResidue', 'MainFeat', 'SecondFeat', 'SecondResidue'],
      dtype='object')

### Select relevant collumns

In [258]:
work2=synonymous[['POS', 'REF', 'ALT', 'AF','type', 'syn',
       'non_syn', 'ref_codon', 'alt_codon', 'ref_residue', 'alt_residue',
       'Genomic_coord_start', 'Genomic_coord_end', 'MainResidue', 'MainFeat',
       'SecondFeat', 'SecondResidue','NbSample','MeanAF']]

In [259]:
work2=work2.drop_duplicates()

In [260]:
workingtable['POS']=workingtable['Genomic_coord_start']
workingtable['REF']='na'
workingtable['AF']='0'
workingtable['type']='NS'

In [261]:
for i,row in workingtable.iterrows():
    if int(row.non_syn)>0:
        workingtable.loc[i, 'type']='S'

Select only non synonymous mutations

In [262]:
workingtable=workingtable[workingtable.type=='S']

In [263]:
work2

,POS,REF,ALT,AF,type,syn,non_syn,ref_codon,alt_codon,ref_residue,alt_residue,Genomic_coord_start,Genomic_coord_end,MainResidue,MainFeat,SecondFeat,SecondResidue,NbSample,MeanAF
2,25563,G,T,1.000000,S,0,1,caG,caT,Q,H,25561,25563,57,ORF3a protein,na,na,90,0.965740
13,23403,A,G,1.000000,S,0,1,gAt,gGt,D,G,23402,23404,614,surface glycoprotein,na,na,195,0.960663
18,21710,T,C,1.000000,S,0,1,Tca,Cca,S,P,21710,21712,50,surface glycoprotein,na,na,1,1.000000
19,20578,G,T,1.000000,S,0,1,Gtg,Ttg,V,L,20578,20580,2371,ORF1b polyprotein,endoRNAse,320,6,0.834473
22,19684,G,T,1.000000,S,0,1,Gta,Tta,V,L,19684,19686,2073,ORF1b polyprotein,endoRNAse,22,23,0.969106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4139,11751,T,C,0.050360,S,0,1,cTc,cCc,L,P,11750,11752,3829,ORF1a polyprotein,nsp6,260,1,0.050360
4141,19159,T,C,0.050314,S,0,1,Tct,Cct,S,P,19159,19161,1898,ORF1b polyprotein,3'-to-5' exonuclease,374,1,0.050314
4142,24131,G,T,0.050279,S,0,1,Ggc,Tgc,G,C,24131,24133,857,surface glycoprotein,na,na,1,0.050279
4143,23903,G,T,0.050279,S,0,1,Gtt,Ttt,V,F,23903,23905,781,surface glycoprotein,na,na,1,0.050279


In [264]:
workingtable

,syn,non_syn,REF,ALT,Genomic_coord_end,Genomic_coord_start,ref_codon,alt_codon,ref_residue,alt_residue,NbSample,MeanAF,MainFeat,MainResidue,SecondFeat,SecondResidue,POS,AF,type
0,0,3,na,0,28312,28310,Cgc,Ggc,R,G,6,0,N,13,NA,0,28310,0,S
1,0,2,na,0,25563,25561,caG,"caA,caT",Q,"Q,H",4,0,ORF3a,57,NA,0,25561,0,S
2,0,4,na,0,1060,1058,acC,acT,T,T,9,0,ORF1a,265,nsp2,85,1058,0,S
3,0,3,na,0,10324,10322,aaG,aaT,K,N,5,0,ORF1a,3353,3C-like proteinase,90,10322,0,S
4,0,3,na,0,11083,11081,ttG,ttT,L,F,9,0,ORF1a,3606,nsp6,37,11081,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,0,1,na,0,6541,6539,0,0,0,"H,Y,",1,0,ORF1a,2092,nsp3,1274,6539,0,S
116,0,1,na,0,9112,9110,0,0,0,"R,H,",3,0,ORF1a,2949,nsp4,186,9110,0,S
121,0,1,na,0,16536,16534,0,0,0,"S,I,G,",9,0,ORF1b,1022,helicase,100,16534,0,S
123,0,1,na,0,17280,17278,0,0,0,"V,L,",4,0,ORF1b,1270,helicase,348,17278,0,S


Concatenate Variant Tables

In [279]:
total=pd.concat([work2,workingtable])

In [280]:
total.POS=total.POS.astype('int64')

In [281]:
total=total.sort_values(by=['POS']).reset_index()

In [282]:
total.head(20)


,index,POS,REF,ALT,AF,type,syn,non_syn,ref_codon,alt_codon,ref_residue,alt_residue,Genomic_coord_start,Genomic_coord_end,MainResidue,MainFeat,SecondFeat,SecondResidue,NbSample,MeanAF
0,3718,269,G,T,0.06087,S,0,1,Gag,Tag,E,*,269,271,2,leader protein,ORF1a polyprotein,2,1,0.060870
1,3471,275,C,T,0.071429,S,0,1,Ctt,Ttt,L,F,275,277,4,leader protein,ORF1a polyprotein,4,1,0.071429
2,3306,285,G,T,0.079545,S,0,1,gGt,gTt,G,V,284,286,7,leader protein,ORF1a polyprotein,7,4,0.137573
3,3020,285,G,T,0.102703,S,0,1,gGt,gTt,G,V,284,286,7,leader protein,ORF1a polyprotein,7,4,0.137573
4,2482,285,G,T,0.29798,S,0,1,gGt,gTt,G,V,284,286,7,leader protein,ORF1a polyprotein,7,4,0.137573
5,3498,285,G,T,0.070064,S,0,1,gGt,gTt,G,V,284,286,7,leader protein,ORF1a polyprotein,7,4,0.137573
6,3386,308,C,A,0.075269,S,0,1,Caa,Aaa,Q,K,308,310,15,leader protein,ORF1a polyprotein,15,2,0.067854
7,3725,308,C,A,0.06044,S,0,1,Caa,Aaa,Q,K,308,310,15,leader protein,ORF1a polyprotein,15,2,0.067854
8,3888,312,T,A,0.055866,S,0,1,cTc,cAc,L,H,311,313,16,leader protein,ORF1a polyprotein,16,1,0.055866
9,3480,350,C,T,0.070539,S,0,1,Cgt,Tgt,R,C,350,352,29,leader protein,ORF1a polyprotein,29,1,0.070539


In [283]:
total['cleanMain']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.MainFeat]]
total['cleanMain']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.cleanMain]]
total['cleanMain']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.cleanMain]]
total['cleanMain']=[re.sub(r"S glycoprotein","S",j) for j in [re.sub(r"membrane",'M',i) for i in total.cleanMain]]

In [284]:
total['cleanMain']

0       leader
1       leader
2       leader
3       leader
4       leader
         ...  
2715     ORF10
2716     ORF10
2717     ORF10
2718     ORF10
2719     ORF10
Name: cleanMain, Length: 2720, dtype: object

In [285]:
total['cleanSecond']=[re.sub(r"membrane glycoprotein","M",j) for j in [re.sub(r" (poly)?protein",'',i) for i in total.SecondFeat]]
total['cleanSecond']=[re.sub(r"RNA-dependent RNA polymerase","RdRp",j) for j in [re.sub(r"nucleocapsid phosphoprotein",'N',i) for i in total.cleanSecond]]
total['cleanSecond']=[re.sub(r"envelope","E",j) for j in [re.sub(r"surface",'S',i) for i in total.cleanSecond]]
total['cleanSecond']=[re.sub(r"membrane",'M',i) for i in total.cleanSecond]

In [286]:
list(set(total.cleanSecond.unique())|set(total.cleanMain.unique()))

['RdRp',
 'nsp2',
 'ORF10',
 'helicase',
 'nsp8',
 'nsp9',
 'ORF7b',
 'nsp11',
 'nsp10',
 'N',
 'ORF1b',
 'nsp7',
 "3'-to-5' exonuclease",
 'leader',
 'nsp6',
 "2'-O-ribose methyltransferase",
 'ORF8',
 'nsp3',
 'endoRNAse',
 'ORF6',
 'ORF1a',
 'NA',
 'ORF3a',
 'E',
 'na',
 'M',
 'nsp4',
 'S',
 'ORF7a',
 '3C-likease']

In [287]:
total.columns

Index(['index', 'POS', 'REF', 'ALT', 'AF', 'type', 'syn', 'non_syn',
       'ref_codon', 'alt_codon', 'ref_residue', 'alt_residue',
       'Genomic_coord_start', 'Genomic_coord_end', 'MainResidue', 'MainFeat',
       'SecondFeat', 'SecondResidue', 'NbSample', 'MeanAF', 'cleanMain',
       'cleanSecond'],
      dtype='object')

In [288]:
total=total[['POS', 'REF', 'ALT', 'AF', 'type', 'syn', 'non_syn',
       'ref_codon', 'alt_codon', 'ref_residue', 'alt_residue',
       'Genomic_coord_start', 'Genomic_coord_end', 'cleanMain', 'MainResidue',
       'cleanSecond', 'SecondResidue','NbSample','MeanAF']]


Create Tables for each Features

In [289]:
total

,POS,REF,ALT,AF,type,syn,non_syn,ref_codon,alt_codon,ref_residue,alt_residue,Genomic_coord_start,Genomic_coord_end,cleanMain,MainResidue,cleanSecond,SecondResidue,NbSample,MeanAF
0,269,G,T,0.06087,S,0,1,Gag,Tag,E,*,269,271,leader,2,ORF1a,2,1,0.060870
1,275,C,T,0.071429,S,0,1,Ctt,Ttt,L,F,275,277,leader,4,ORF1a,4,1,0.071429
2,285,G,T,0.079545,S,0,1,gGt,gTt,G,V,284,286,leader,7,ORF1a,7,4,0.137573
3,285,G,T,0.102703,S,0,1,gGt,gTt,G,V,284,286,leader,7,ORF1a,7,4,0.137573
4,285,G,T,0.29798,S,0,1,gGt,gTt,G,V,284,286,leader,7,ORF1a,7,4,0.137573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2715,29612,T,C,0.056738,S,0,1,Tgc,Cgc,C,R,29612,29614,ORF10,19,na,na,1,0.056738
2716,29625,C,T,0.981366,S,0,1,tCt,tTt,S,F,29624,29626,ORF10,23,na,na,1,0.981366
2717,29642,C,T,0.054217,S,0,1,Caa,Taa,Q,*,29642,29644,ORF10,29,na,na,3,0.316471
2718,29642,C,T,0.796296,S,0,1,Caa,Taa,Q,*,29642,29644,ORF10,29,na,na,3,0.316471


In [290]:
dicfeatures={}
for i in list(set(total.cleanSecond.unique())|set(total.cleanMain.unique())) :
    dicfeatures[i]=total[(total.cleanMain==i) | (total.cleanSecond==i)]

In [291]:
dicfeatures.keys()

dict_keys(['RdRp', 'nsp2', 'ORF10', 'helicase', 'nsp8', 'nsp9', 'ORF7b', 'nsp11', 'nsp10', 'N', 'ORF1b', 'nsp7', "3'-to-5' exonuclease", 'leader', 'nsp6', "2'-O-ribose methyltransferase", 'ORF8', 'nsp3', 'endoRNAse', 'ORF6', 'ORF1a', 'NA', 'ORF3a', 'E', 'na', 'M', 'nsp4', 'S', 'ORF7a', '3C-likease'])

In [292]:
for i in dicfeatures.keys():
    dicfeatures[i].to_csv(i+'_var.csv',  sep='\t',index=False, header=True)